<a href="https://colab.research.google.com/github/memonmaroof06/neural-network-challenge-2/blob/main/M19_Starter_Code/M19_Starter_Code/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [42]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [43]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [44]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [45]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
    'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
    'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print("X_df data types:")
print(X_df.dtypes)


X_df data types:
Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [46]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [51]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
overtime_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
overtime_encoded_train = overtime_encoder.fit_transform(X_train[['OverTime']])
overtime_encoded_test = overtime_encoder.transform(X_test[['OverTime']])

X_train_numeric = X_train.drop('OverTime', axis=1)
X_test_numeric = X_test.drop('OverTime', axis=1)

In [52]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train_numeric)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test_numeric)



In [54]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoded_train = dept_encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_test = dept_encoder.transform(y_test[['Department']])



In [55]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoded_train = attr_encoder.fit_transform(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attr_encoded_test = attr_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [56]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]
print(f"Number of input features: {input_dim}")
# Create the input layer
inputs = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(inputs)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

Number of input features: 9


In [57]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [58]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [59]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])


# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 9)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,381 (67.89 KB)

 Trainable params: 17,381 (67.89 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Train the model
history = model.fit(
    X_train_scaled,
    [dept_encoded_train, attr_encoded_train],
    validation_split=0.2,
    epochs=100,
    batch_size=32 )



Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - attrition_output_accuracy: 0.8082 - department_output_accuracy: 0.6222 - loss: 1.4728 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.3233
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8397 - department_output_accuracy: 0.6670 - loss: 1.2191 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.2962
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8484 - department_output_accuracy: 0.6426 - loss: 1.1951 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.2973
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8479 - department_output_accuracy: 0.6697 - loss: 1.1280 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6356 - val_loss: 1.2863
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [68]:
# Evaluate the model with the testing data
results = model.evaluate(
    X_test_scaled,
    [dept_encoded_test, attr_encoded_test],
    verbose=1,
     return_dict=True
)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8234 - department_output_accuracy: 0.5448 - loss: 4.2678 


In [69]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {results['department_output_accuracy']}")
print(f"Attrition predictions accuracy: {results['attrition_output_accuracy']}")

Department predictions accuracy: 0.5714285969734192
Attrition predictions accuracy: 0.8333333134651184


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. it is not becuase ecuase the dataset shows class imbalance, also ROC curves and AUC scores would better handle class imbalnce.
2. softmax activation becuase its a multi class classification also its the best option for mutually categories.
for attribution out put sigmoid becuase sigmoid outputs probability between 0 and 1 .
3. techniques to handle class imbalnce like class weights in model training , also create interations terms , try diffrent layer sizes , early stopping